# Clean and Explore Stock Information

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline

In [3]:
# due to file size, set low_memory to False to alter load order
nasdaq = pd.read_csv("./data/nasdaq_csv.csv",index_col=0, low_memory=False)
nyse = pd.read_csv("./data/nyse_csv.csv",index_col=0, low_memory=False)

### Check the size of both of our datasets

In [4]:
nasdaq.shape, nyse.shape

((8752326, 8), (6994408, 8))

Similar shapes, first noticeable difference is the row count difference. We will confirm below, but one possibility is the NASDAQ dataset goes further back in time despite NYSE having the longer tenure.

In [5]:
nasdaq.head(2)

,Date,Low,Open,Volume,High,Close,Adjusted Close,ticker
0,16-02-1990,0.073785,0.0,940636800.0,0.0798610001802444,0.077257,0.054863,CSCO
1,20-02-1990,0.074653,0.0,151862400.0,0.0798610001802444,0.079861,0.056712,CSCO


In [6]:
nyse.head(2)

,Date,Low,Open,Volume,High,Close,Adjusted Close,ticker
0,19-06-1992,15.0,15.0,86000.0,15.0,15.0,3.640341,NXN
1,22-06-1992,15.0,15.0,17000.0,15.0,15.0,3.640341,NXN


Let's check column types, assuming Date is currently 'object' across both dataframes as pandas has some difficulty recognizing datetime

In [7]:
# nasdaq.info()

In [8]:
# nyse.info()

Both datasets have Date as object, but there are some other surprises:
- we have a small null population
- Open, High, of our NASDAQ set is object and not float

We'll take care of the nulls first then take care of our column types

In [9]:
nasdaq.isnull().sum()

Date                   0
Low               130276
Open              130276
Volume            130276
High              130277
Close             130277
Adjusted Close    130277
ticker                 0
dtype: int64

In [10]:
print(f"Percentage of NASDAQ Null rows: {round(((130_277/8_752_326)*100),2)}%") 

Percentage of NASDAQ Null rows: 1.49%


In [11]:
nyse.isnull().sum()

Date                  0
Low               94982
Open              94982
Volume            94982
High              94982
Close             94982
Adjusted Close    94982
ticker                0
dtype: int64

In [12]:
print(f"Percentage of NYSE Null rows: {round(((94_982/6_994_408)*100),2)}%") 

Percentage of NYSE Null rows: 1.36%


The population of rows of null values within each dataset respectively is less than 5% of the entire data set. We have a couple of options:
1. Drop the null rows as the total percentage is within accept range
2. Fill the null values using simple means such as fillna mean, mode, back or forward fill
3. Fill the null values using regression
4. Integrate with API and fill in the information missing by date

For our first iteration we are going to simply drop the nulls to save time. After we build and test our data pipeline for both Linear Regression for stock price prediction and ARIMA for Time Series modeling, we can return to this if we think it can improve our models.


In [13]:
nasdaq.dropna(axis=0, inplace=True)

In [14]:
nyse.dropna(axis=0, inplace=True)

In [15]:
nasdaq.shape, nyse.shape

((8622049, 8), (6899426, 8))

Change Date to Datetime and we are ready to start looking through descriptive statistics, distributions of data, and check for any seasonality in the data

In [16]:
# remove an additional bad row from nasdaq that looks like human error or merge error from dataset
nasdaq = nasdaq[nasdaq["Date"]!="18-1218-12-1991"]

In [17]:
# add format to speed up performance on large dataset
nasdaq["Date"] = pd.to_datetime(nasdaq['Date'],format="%d-%m-%Y")
nyse["Date"] = pd.to_datetime(nyse['Date'],format="%d-%m-%Y")

Change Open, High to float64

After trying to .astype() Open and High, we found that there was some misplaced dates in those fields which caused the incorrect type classification.

We created a small convert function to try to change the value, and if not return None.

Afterwards we will drop nulls to continue

In [50]:
def convert(value):
    try:
        return float(value)
    except:
        return None

In [48]:
nasdaq["Open"] = nasdaq["Open"].apply(lambda x: convert(x))
nasdaq["High"] = nasdaq["High"].apply(lambda x: convert(x))

In [49]:
nasdaq.isnull().sum()

Date              0
Low               0
Open              2
Volume            0
High              1
Close             0
Adjusted Close    0
ticker            0
dtype: int64

In [51]:
nasdaq.dropna(inplace=True)

In [53]:
nasdaq.isnull().sum().sum()

0

# Exploratory Data Analysis

How many individual stocks exist within our NASDAQ and NYSE datasets?

In [54]:
len(nasdaq["ticker"].unique())

1564

In [55]:
len(nyse["ticker"].unique())

1145

There are more individual tickers in our NASDAQ set than our NYSE

In [63]:
nasdaq.describe().applymap("{0:.5f}".format).T

,count,mean,std,min,25%,50%,75%,max
Low,8622045.00000,4215774634.95771,434712007664.18317,0.00000,5.20000,13.08000,28.59000,85239003086848.00000
Open,8622045.00000,4324769387.52614,445854127743.85724,0.00000,5.03919,13.15000,28.93000,91249197449216.00000
Volume,8622045.00000,1486638.44309,18432870.91560,0.00000,6315.00000,57652.00000,337300.00000,7421640800.00000
High,8622045.00000,4411168149.48307,454139004931.33789,0.00000,5.45000,13.55000,29.56500,91249197449216.00000
Close,8622045.00000,4306318288.94347,443797023123.95166,0.00000,5.32000,13.31250,29.07000,86750999347200.00000
Adjusted Close,8622045.00000,4306318102.90701,443797023126.97766,-101.84761,4.18140,10.60026,25.32291,86750999347200.00000


In [62]:
nyse.describe().applymap("{0:.5f}".format).T

,count,mean,std,min,25%,50%,75%,max
Low,6899426.00000,720.45457,57883.54953,0.00001,9.25000,16.75000,34.84000,12942857.00000
Open,6899426.00000,749.53391,60739.65928,0.00000,8.95000,16.68500,35.00000,14285714.00000
Volume,6899426.00000,1094741.40704,4033555.99712,0.00000,37265.00000,174400.00000,785500.00000,1222342500.00000
High,6899426.00000,781.54799,63731.55198,0.00001,9.49000,17.14000,35.72000,14728571.00000
Close,6899426.00000,750.20028,60739.28341,0.00001,9.37500,16.95000,35.28000,14285714.00000
Adjusted Close,6899426.00000,742.16821,60739.33822,-0.37842,4.21782,10.75000,26.58941,14285714.00000
